In [79]:
import numpy as np
import pandas as pd
import os

from scipy import stats

os.chdir('/home/galina/PycharmProjects/Peptide Signal')

# Helper function, finds weight of aminoacid

In [80]:
def weight(name, dictionary):
    w = 0
    for e in name:
        w = w + dictionary[e]
    return int(w)

# Aminoacid weights to dictionary

In [81]:
amino = pd.read_csv('Aminoacids.csv', header = 0)
amino.head()

,Aminoacid,1-Letter Code,"Molecular weight, kDa"
0,Alanine,A,89
1,Arginine,R,174
2,Asparagine,N,132
3,Aspartic,D,133
4,Cysteine,C,121


In [82]:
amino.drop('Aminoacid', axis = 1, inplace = True)
amino.columns = ['Code', 'Weight']
weights = amino.set_index('Code')['Weight'].to_dict()
weights

{nan: nan,
 'A': 89.0,
 'C': 121.0,
 'D': 133.0,
 'E': 146.0,
 'F': 165.0,
 'G': 75.0,
 'H': 155.0,
 'I': 131.0,
 'K': 146.0,
 'L': 131.0,
 'M': 149.0,
 'N': 132.0,
 'P': 115.0,
 'Q': 147.0,
 'R': 174.0,
 'S': 105.0,
 'T': 119.0,
 'V': 117.0,
 'W': 204.0,
 'Y': 181.0}

# Peptide data cleaning

In [106]:
data = pd.read_csv('Peptides.csv', header = 0)

data.drop(['Grid Index',
           'Peptide Signal Uniformity',
           'Streptavidin Signal Uniformity',
           
#            'Streptavidin Association Signal (Saturated)',
#            'Streptavidin Dissociation Signal (Saturated)',
           
#            'Tag',
#            'Alignment',
#            'Empty',
#            'Thrombin',
#            'PSA',
#            'StrepTag', 
#            'Myc', 
#            'FLAG',
#            'NA', 
#            'Epitope', 
#            'Bad Washed',
#            'Bad Strep',
           
           'aMyc Signal Uniformity'], axis = 1, inplace = True)

# Add 'Bad' column
bad_indices = [int(x) for x in np.genfromtxt("bad spots.txt", delimiter="\n")]
binary_bad = [0] * len(data)
for i in bad_indices:
    binary_bad[i] = 1

data['Bad'] = binary_bad    

data.head()

,Peptide,Peptide Length,Peptide Signal,Streptavidin Signal,aMyc Signal,aMyc Association Signal,aMyc Dissociation Signal,Bad
0,WTHPQFAT,8,0.000642,0.001180,-0.000015,0.000086,0.000006,0
1,WTHPQFAT,8,0.001130,0.001210,-0.000013,-0.000195,-0.000060,0
2,WTHLQFAT,8,0.000244,0.000022,0.000058,0.000019,-0.000035,0
3,NGQFQVWIPLAQK,13,0.002490,-0.000024,0.000177,0.000547,0.000392,0
4,LQLHPQAGK,9,0.003110,0.000408,0.000104,0.000086,0.000095,0


In [107]:
# remove NaNs, Xs and bad spots
data = data.dropna()
data = data[ data['Peptide'] != 'X' ]
data = data[ data['Bad'] == 0 ]

data.sort('Peptide', inplace = True)
data = data.reset_index(drop = True)

data.head()

,Peptide,Peptide Length,Peptide Signal,Streptavidin Signal,aMyc Signal,aMyc Association Signal,aMyc Dissociation Signal,Bad
0,AGQFQVWIPGAQK,13,0.00253,0.000011,0.000092,0.000125,0.000068,0
1,AGQFQVWIPGAQK,13,0.00279,-0.000060,0.000054,0.000130,0.000026,0
2,AGQFQVWIPGAQK,13,0.00336,-0.000017,0.000147,0.000193,0.000205,0
3,AGQFQVWIPGAQK,13,0.00255,-0.000044,0.000129,0.000052,0.000090,0
4,AGQFQVWIPGAQK,13,0.00187,0.000028,0.000053,0.000144,0.000139,0


# Calculate mean and standard deviation by peptide

In [108]:
grouped = data.groupby(['Peptide',
                        'Peptide Length'])                       

f = {'Peptide Signal'           : [np.mean, np.std, stats.sem, len],
     'Streptavidin Signal'      : [np.mean, np.std, stats.sem],
     'aMyc Signal'              : [np.mean, np.std, stats.sem],
     'aMyc Association Signal'  : [np.mean, np.std, stats.sem],
     'aMyc Dissociation Signal' : [np.mean, np.std, stats.sem]}     

In [110]:
result = grouped.agg(f)

# result.columns = result.columns.droplevel()
result.reset_index(inplace = True)

result.columns = ['Peptide',
                  'Peptide Length',    
                  
                  'Streptavidin Signal Mean',
                  'Streptavidin Signal SD',
                  'Streptavidin Signal SE',

                  'aMyc Dissociation Signal Mean',
                  'aMyc Dissociation Signal SD',
                  'aMyc Dissociation Signal SE',

                  'aMyc Association Signal Mean',
                  'aMyc Association Signal SD',
                  'aMyc Association Signal SE', 
                    
                  'aMyc Signal Mean',
                  'aMyc Signal SD',
                  'aMyc Signal SE',
                   
                  'Peptide Signal Mean',
                  'Peptide Signal SD',
                  'Peptide Signal SE',
                  
                  'Number of Spots']


result['Streptavidin Signal Mean NN'] = abs( result['Streptavidin Signal Mean']
                                          * (result['Streptavidin Signal Mean'] > 0) )
result['Peptide Signal Mean NN'] = abs( result['Peptide Signal Mean']
                                     * (result['Peptide Signal Mean'] > 0) )
result['aMyc Signal Mean NN'] = abs( result['aMyc Signal Mean']
                                  * (result['aMyc Signal Mean'] > 0) )
result['aMyc Association Signal Mean NN'] = abs( result['aMyc Association Signal Mean']
                                              * (result['aMyc Association Signal Mean'] > 0) )
result['aMyc Dissociation Signal Mean NN'] = abs( result['aMyc Dissociation Signal Mean']
                                               * (result['aMyc Dissociation Signal Mean'] > 0) )

result.head()

,Peptide,Peptide Length,Streptavidin Signal Mean,Streptavidin Signal SD,Streptavidin Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal SD,...,aMyc Signal SE,Peptide Signal Mean,Peptide Signal SD,Peptide Signal SE,Number of Spots,Streptavidin Signal Mean NN,Peptide Signal Mean NN,aMyc Signal Mean NN,aMyc Association Signal Mean NN,aMyc Dissociation Signal Mean NN
0,AGQFQVWIPGAQK,13,-0.000002,0.000043,0.000010,0.000165,0.000081,0.000018,0.000205,0.000091,...,0.000011,0.002625,0.000342,0.000077,20,0.000000,0.002625,0.000129,0.000205,0.000165
1,AISPPPK,7,-0.000018,0.000051,0.000005,0.000135,0.000118,0.000012,0.000110,0.000132,...,0.000004,0.002833,0.000502,0.000051,98,0.000000,0.002833,0.000097,0.000110,0.000135
2,AKFPIPLGKQSG,12,-0.000031,0.000030,0.000015,0.000207,0.000037,0.000018,0.000207,0.000091,...,0.000020,0.002910,0.000384,0.000192,4,0.000000,0.002910,0.000105,0.000207,0.000207
3,AQWHPQAGK,9,0.000783,0.000087,0.000043,0.000101,0.000077,0.000038,0.000060,0.000077,...,0.000007,0.002933,0.000310,0.000155,4,0.000783,0.002933,0.000047,0.000060,0.000101
4,ATHPQFAT,8,0.000987,0.000136,0.000068,-0.000053,0.000066,0.000033,-0.000132,0.000143,...,0.000017,0.001607,0.000362,0.000181,4,0.000987,0.001607,0.000000,0.000000,0.000000


# Add weight, and sort

In [111]:
result['Peptide Weight'] = result['Peptide'].map(lambda x: weight(x, weights))
result.sort(['Peptide Length', 'Peptide'], inplace = True)
result = result.reset_index(drop = True)

result.head()

,Peptide,Peptide Length,Streptavidin Signal Mean,Streptavidin Signal SD,Streptavidin Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal SD,...,Peptide Signal Mean,Peptide Signal SD,Peptide Signal SE,Number of Spots,Streptavidin Signal Mean NN,Peptide Signal Mean NN,aMyc Signal Mean NN,aMyc Association Signal Mean NN,aMyc Dissociation Signal Mean NN,Peptide Weight
0,AISPPPK,7,-0.000018,0.000051,0.000005,0.000135,0.000118,0.000012,0.000110,0.000132,...,0.002833,0.000502,0.000051,98,0.000000,0.002833,0.000097,0.000110,0.000135,816
1,THPQFAT,7,0.000742,0.000101,0.000058,0.000008,0.000055,0.000032,0.000024,0.000063,...,0.001647,0.000257,0.000148,3,0.000742,0.001647,0.000000,0.000024,0.000008,909
2,WHPQFAT,7,0.000610,0.000051,0.000025,-0.000016,0.000094,0.000047,-0.000004,0.000068,...,0.000924,0.000134,0.000067,4,0.000610,0.000924,0.000000,0.000000,0.000000,994
3,WTHPFAT,7,-0.000002,0.000057,0.000029,-0.000047,0.000063,0.000032,-0.000158,0.000073,...,0.001179,0.000203,0.000101,4,0.000000,0.001179,0.000013,0.000000,0.000000,966
4,WTHPQAT,7,0.000214,0.000045,0.000026,0.000001,0.000168,0.000097,-0.000053,0.000216,...,0.001800,0.000416,0.000240,3,0.000214,0.001800,0.000000,0.000000,0.000001,948


In [112]:
# rearrange order of columns
result = result.reindex_axis(['Peptide',
                              'Number of Spots',
                              'Peptide Length',
                              'Peptide Weight',
                              
                              'Peptide Signal Mean',
                              'Peptide Signal Mean NN',
                              'Peptide Signal SD',
                              'Peptide Signal SE',
                              
                              'Streptavidin Signal Mean',
                              'Streptavidin Signal Mean NN',
                              'Streptavidin Signal SD',
                              'Streptavidin Signal SE',
                              
                              'aMyc Signal Mean',
                              'aMyc Signal Mean NN',
                              'aMyc Signal SD',
                              'aMyc Signal SE',              
                              
                              'aMyc Dissociation Signal Mean',
                              'aMyc Dissociation Signal Mean NN',
                              'aMyc Dissociation Signal SD',
                              'aMyc Dissociation Signal SE',

                              'aMyc Association Signal Mean',
                              'aMyc Association Signal Mean NN',
                              'aMyc Association Signal SD',
                              'aMyc Association Signal SE'], axis=1)
result.head()

,Peptide,Number of Spots,Peptide Length,Peptide Weight,Peptide Signal Mean,Peptide Signal Mean NN,Peptide Signal SD,Peptide Signal SE,Streptavidin Signal Mean,Streptavidin Signal Mean NN,...,aMyc Signal SD,aMyc Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal Mean NN,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal Mean NN,aMyc Association Signal SD,aMyc Association Signal SE
0,AISPPPK,98,7,816,0.002833,0.002833,0.000502,0.000051,-0.000018,0.000000,...,0.000042,0.000004,0.000135,0.000135,0.000118,0.000012,0.000110,0.000110,0.000132,0.000013
1,THPQFAT,3,7,909,0.001647,0.001647,0.000257,0.000148,0.000742,0.000742,...,0.000007,0.000004,0.000008,0.000008,0.000055,0.000032,0.000024,0.000024,0.000063,0.000037
2,WHPQFAT,4,7,994,0.000924,0.000924,0.000134,0.000067,0.000610,0.000610,...,0.000024,0.000012,-0.000016,0.000000,0.000094,0.000047,-0.000004,0.000000,0.000068,0.000034
3,WTHPFAT,4,7,966,0.001179,0.001179,0.000203,0.000101,-0.000002,0.000000,...,0.000032,0.000016,-0.000047,0.000000,0.000063,0.000032,-0.000158,0.000000,0.000073,0.000036
4,WTHPQAT,3,7,948,0.001800,0.001800,0.000416,0.000240,0.000214,0.000214,...,0.000012,0.000007,0.000001,0.000001,0.000168,0.000097,-0.000053,0.000000,0.000216,0.000125


In [113]:
result.to_csv('result.csv', index = False)